# Advent of Code 2020, Python edition

Annotated solutions in Python.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [138]:
from collections import Counter, defaultdict, deque
from dataclasses import dataclass
from functools import lru_cache, reduce
from itertools import accumulate, chain, combinations, count, product
from math import ceil, prod
from operator import mul
import re
import sys

### Day 1: Report Repair
https://adventofcode.com/2020/day/1

In [20]:
def day01(d, size):
    def finder(x): return sum(x) == 2020
    return prod(next(filter(finder, combinations(d, size))))

with open('data/2020/day01.txt') as f:
    data = [int(l) for l in f]

p1 = day01(data, 2)
p2 = day01(data, 3)

assert p1 == 73371
assert p2 == 127642310
print(p1, p2)

73371 127642310


### Day 2: Password Philosophy
https://adventofcode.com/2020/day/2

In [21]:
def part1(d):
    count = sum(map(lambda c: c == d[2], d[-1]))
    return int(d[0]) <= count <= int(d[1])

def part2(d):
    return (d[2] == d[-1][int(d[0])-1]) != (d[2] == d[-1][int(d[1])-1])

with open('data/2020/day02.txt') as f:
    data = [re.split(r'[ :-]', l) for l in f]

p1 = sum(map(part1, data))
p2 = sum(map(part2, data))

assert p1 == 528
assert p2 == 497
print(p1, p2)

528 497


### Day 3: Toboggan Trajectory
https://adventofcode.com/2020/day/3

In [22]:
def day03(d, dy, dx):
    return sum('#'==d[dy*i][(dx*i)%len(d[0])] for i in range(ceil(len(d)/dy)))

with open('data/2020/day03.txt') as f:
    data = f.read().splitlines()

result = [day03(data, *v) for v in [[1, 1], [1, 3], [1, 5], [1, 7], [2, 1]]]

p1 = result[1]
p2 = prod(result)

assert p1 == 203
assert p2 == 3316272960
print(p1, p2)

203 3316272960


### Day 4: Passport Processing
https://adventofcode.com/2020/day/4

In [32]:
SYMTAB = {
    'eyr': lambda a: 2020 <= int(a) <= 2030,
    'iyr': lambda a: 2010 <= int(a) <= 2020,
    'byr': lambda a: 1920 <= int(a) <= 2002,
    'ecl': lambda a: a in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'},
    'cid': lambda a: True,
    'pid': lambda a: re.match(r'^\d{9}$', a),
    'hcl': lambda a: re.match(r'^#[a-f0-9]{6}$', a),
    'hgt': lambda a: re.match(r'^(((59|6[0-9]|7[0-6])in)|((1[5-8][0-9]|19[0-3])cm))$',a)
}

def part1(pp):
    return {'byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'} <= set(pp.keys())

def part2(passports):
    count = 0
    for pp in passports:
        try:
            for check, arg in pp.items():
                if not (part1(pp) and SYMTAB[check](arg)):
                    raise Exception()
            count += 1
        except:        # Sorry. Convenience wins over pythonic
            pass

    return count

def parse_data(data):
    keyvals = {}
    for d in (re.findall(r'([^:\s]+):([^\s]+)', line) for line in data):
        if not d:
            yield keyvals
            keyvals = {}
            continue
        keyvals |= dict(d) # Recent pythons only
    yield keyvals

with open('data/2020/day04.txt') as f:
    passports = list(parse_data(f))

p1 = sum(part1(p) for p in passports)
p2 = part2(passports)

assert p1 == 256
assert p2 == 198
print(p1, p2)

256 198


### Day 5: Binary Boarding
https://adventofcode.com/2020/day/5

In [24]:
def SeatID(spec):
    return int(''.join({'F':'0','L':'0'}.get(l, '1') for l in spec), 2)

with open('data/2020/day05.txt') as f:
    data = [SeatID(l) for l in f.read().splitlines()]

p1 = max(data)
p2 = list(set(range(min(data), p1)) - set(data))[0]

assert p1 == 888
assert p2 == 522
print(p1, p2)

888 522


### Day 6: Custom Customs
https://adventofcode.com/2020/day/6

In [25]:
def gr(data):
    g = []
    for l in data:
        if l != '':
            g.append(l)
        else:
            yield g
            g = []
    yield g
    
with open('data/2020/day06.txt') as f:
    data = list(gr(f.read().splitlines()))

p1 = sum((len(set(''.join(l))) for l in data))
p2 = sum(len(set.intersection(*[set(s) for s in g])) for g in data)

assert p1 == 6416
assert p2 == 3050
print(p1, p2)

6416 3050


### Day 7: Handy Haversacks
https://adventofcode.com/2020/day/7

In [26]:
def parse(data):
    head, tail = data.split(' bags contain ')
    tail = tail.replace('no other bags.', '0 other bags.')
    contents = [
        re.findall(r'^(\d+)\s+(\w+\s\w+)', ss) for ss in tail.split(', ')
    ]
    return (head, [(int(w[0][0]), w[0][1]) for w in contents])

def invert(data):
    contained_in = defaultdict(set)
    for spec in data:
        for (_, name) in spec[1]:
            contained_in[name].add(spec[0])
    return contained_in

def vert(data):
    contains = defaultdict(dict)
    for spec in data:
        for (mag, name) in spec[1]:
            contains[spec[0]][name] = mag
    return contains

def part1(inverted):
    found = set()
    queue = list(inverted['shiny gold'])
    for item in queue:
        found.add(item)
        if item in inverted:
            queue.extend(list(inverted[item]))
    return len(found)

def part2(verted, key='shiny gold'):
    if key == 'other bags': return 0
    return sum(vv * (1 + part2(verted, kk)) for kk, vv in verted[key].items())

with open('data/2020/day07.txt') as f:
    data = [parse(l) for l in f]

p1 = part1(invert(data))
p2 = part2(vert(data))

assert p1 == 185
assert p2 == 89084
print(p1, p2)

185 89084


### Day 8: Handheld Halting
https://adventofcode.com/2020/day/8

In [27]:
@dataclass
class State:
    ip: int
    code: list
    args: list
    acc: int
    lines: list

    def halted(self):
        return self.ip >= len(self.code)
    
    def infloop(self):
        return 2 in self.lines
    
    def step(self):
        instr = self.code[self.ip]
        arg = self.args[self.ip]
        self.lines[self.ip] += 1
        if self.infloop():
            return 
        if instr == 'nop':
            self.ip += 1
        elif instr == 'acc':
            self.acc += arg
            self.ip += 1
        else:
            self.ip += arg
            
    def run(self):
        while not self.halted() and not self.infloop():
            self.step()
        return self.acc

def tweak(code):
    def repl(data, idx, to):
        data[idx] = to
        return data

    for idx, instr in enumerate(code):
        if instr == 'nop':
            yield repl(list(code), idx, 'jmp')
        elif instr == 'jmp':
            yield repl(list(code), idx, 'nop')

def part2(data):
    for code in tweak(data[0]):
        state = State(0, code, [int(a) for a in data[1]], 0, [0]*len(code))
        result = state.run()
        if state.halted():
            return result

with open('data/2020/day08.txt') as f:
    data = list(zip(*[l.split() for l in f]))

part1 = State(0, data[0], [int(a) for a in data[1]], 0, [0]*len(data[0]))

p1 = part1.run()
p2 = part2(data)

assert p1 == 1134
assert p2 == 1205
print(p1, p2)

1134 1205


### Day 9: Encoding Error
https://adventofcode.com/2020/day/9

For part 2, the sum of a stretch is the sum to the end, minus sum to the start. Outer product gives us all stretches, but we need to convert from single index back to start and end of the stretch. In APL, the outer product would have given us a 2D matrix of the correct dimensions, rather than a vector.

In [45]:
def find(data, idx, size):
    return data[idx] not in map(sum, combinations(data[idx-size:idx], 2))
    
def part1(data, win):
    for idx in range(win, len(data)):
        if find(data, idx, win): return data[idx]
    return -1

def part2(data, item):
    sums = list(accumulate(data))
    idx = [a[0]-a[1] for a in product(sums, sums)].index(item)
    values = data[1+idx%len(sums):idx//len(sums)+2]
    return min(values) + max(values)
        
with open('data/2020/day09.txt') as f:
    data = list(map(int, f))

p1 = part1(data, 25)
p2 = part2(data, p1)

assert p1 == 1639024365
assert p2 == 219202240
print(p1, p2)

1639024365 219202240


### Day 10: Adapter Array
https://adventofcode.com/2020/day/10

Part 2 -- a [Dynamic Programming](https://en.wikipedia.org/wiki/Dynamic_programming) solution. The cache only needs two slots.

In [113]:
def part1(data):
    diff = [data[i]-data[i-1] for i in range(1, len(data))] # Windowed reduction
    return prod(sum(d==c for d in diff) for c in [1, 3])

@lru_cache(maxsize=2)
def part2(v, data):
    if not data: return 1
    count = 0       # Valid solutions from this point
    for idx, item in enumerate(data):
        if item - v <= 3:
            count += part2(item, tuple(data[1+idx:]))
    return count

with open('data/2020/day10.txt') as f:
    data = sorted(list(map(int, f)))
data = [0, *data, 3 + data[-1]]

p1 = part1(data)
p2 = part2(0, tuple(data[1:]))

assert p1 == 2059
assert p2 == 86812553324672
print(p1, p2)

2059 86812553324672


Here's part 2 as a single reduction. Python folds left-right, unlike APL.

In [114]:
with open('data/2020/day10.txt') as f:
    data = sorted(list(map(int, f)))                # (⍋⌷¨⊂)⍎⍕⊃⎕NGET'data/2020/day10.txt'1

p2 = reduce(
    lambda acc, w:[*acc[1:], w*sum(acc)],           # {1↓⍵,⍺×+/⍵}
    [int(i in data) for i in range(1, data[-1]+1)], # ⌽data∊⍨1+⍳⊃⌽data
    [0, 0, 1]                                       # ⊂0 0 1
)[2]

assert p2 == 86812553324672
print(p2)

86812553324672


### Day 11: Seating System
https://adventofcode.com/2020/day/11

In [115]:
OFFSET = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))

def N8(data, pos):
    coords = ((pos[0] + t[0], pos[1] + t[1]) for t in OFFSET)
    return [
        data[y][x] for y, x in coords
        if 0 <= y < len(data) and 0 <= x < len(data[0])
    ]

def Transition(data, pos):
    elem = data[pos[0]][pos[1]]
    if elem == '.': return '.'
    c = Counter(N8(data, pos))
    if elem == 'L' and '#' not in c: return '#'
    if elem == '#' and c['#'] >= 4: return 'L'
    return elem

def part1(data):
    return tuple(
        tuple(Transition(data, (y, x)) for x in range(len(data[0])))
        for y in range(len(data)))

def C(data): return sum(e == '#' for e in chain(*data))

with open('/Users/stefan/work/dyalog/AoCDyalog/data/2020/day11.txt') as f:
    data = tuple(tuple(l) for l in f.read().splitlines())

while (new := part1(data)) != data:
    data = new

p1 = C(data)

assert p1 == 2344
print(p1)

# TODO: Part 2

2344


### Day 12: Rain Risk
https://adventofcode.com/2020/day/12

In [135]:
# ---- PART 1 --------
def N(acc, mag): return [acc[0] - mag, acc[1], acc[2]]
def S(acc, mag): return N(acc, -mag)
def E(acc, mag): return [acc[0], acc[1] + mag, acc[2]]
def W(acc, mag): return E(acc, -mag)

def R(acc, mag):
    acc[2].rotate(-mag // 90)
    return [*acc[:2], acc[2]]

def L(acc, mag): return R(acc, -mag)

def F(acc, mag):
    dv = ((0, mag), (mag, 0), (0, -mag), (-mag, 0))[acc[2][0]]
    return [dv[0] + acc[0], dv[1] + acc[1], acc[2]]

def run(acc, item):
    return {'N': N, 'W': W, 'E': E, 'S': S, 'L': L, 'R': R, 'F': F,}[item[0]](acc, item[1])

# ---- PART 2 --------
def N2(acc, mag): return [acc[0]-mag, acc[1], acc[2], acc[3]]
def S2(acc, mag): return N2(acc, -mag)
def E2(acc, mag): return [acc[0], acc[1]+mag, acc[2], acc[3]]
def W2(acc, mag): return E2(acc, -mag)
def F2(acc, mag):
    d = [acc[0]*mag, acc[1]*mag]
    return [acc[0], acc[1], acc[2]+d[0], acc[3]+d[1]]
                          
def R2(acc, mag):
    if mag == 90:
        return [acc[1], -acc[0], acc[2], acc[3]]
    if mag == 180:
        return [-acc[0], -acc[1], acc[2], acc[3]]
    return [-acc[1], acc[0], acc[2], acc[3]]

def L2(acc, mag):
    return R2(acc, 360-mag)

def run2(acc, item):
    return {'N': N2, 'W': W2, 'E': E2, 'S': S2, 'L': L2, 'R': R2, 'F': F2,}[item[0]](acc, item[1])

# -------------------
with open('data/2020/day12.txt') as f:
    (cmd, arg) = list(zip(*[m[0] for m in [re.findall(r'^([FRLNWES])(\d+)$', l) for l in f]]))

data = tuple(zip(cmd, tuple(map(int, arg))))

result = list(reduce(run, data, [0, 0, deque([0, 1, 2, 3])]))[:2]
p1 = abs(result[0]) + abs(result[1])

result = list(reduce(run2, data, [-1, 10, 0, 0])) 
                                 
p2 = abs(result[2]) + abs(result[3])

assert p1 == 2879
assert p2 == 178986

print(p1, p2)

2879 178986


### Day 13: Shuttle Search
https://adventofcode.com/2020/day/13

Python implementation of the CRT taken verbatim from

https://rosettacode.org/wiki/Chinese_remainder_theorem#Python

In [139]:
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a * b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod

def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

with open('data/2020/day13.txt') as f:
    data = f.read().splitlines()

timestamp = int(data[0])
buses = eval(f"[{data[1].replace('x', '0')}]")
n = [i for i in buses if i != 0]

p1 = next(delta * bus for delta in count() for bus in n
          if (timestamp + delta) % bus == 0)

idx = [buses.index(i) for i in n]
a = [n[j] - idx[j] + 1 for j in range(len(n))]
p2 = chinese_remainder(n, a) - 1

assert p1 == 410
assert p2 == 600691418730595
print(p1, p2)

410 600691418730595
